In [1]:
import string
import random
import time

In [2]:
from fake_useragent import UserAgent
user_agent = UserAgent().random

In [3]:
import wikipediaapi

wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='user_agent'
)

page_title = 'M S Dhoni'

page = wiki.page(page_title)

text_content = page.text

f = "msd.txt"
with open(f, 'w', encoding='utf-8') as file:
    file.write(text_content)


In [4]:
def tokenize(text):
    for punct in string.punctuation:
        text = text.replace(punct, ' '+punct+' ')
    t = text.split()
    return t

In [5]:
def get_ngrams(n, tokens):
    ngrams = []
    tokens = (n-1)*['<START>']+tokens
    for i in range(n - 1, len(tokens)):
        context = tuple(tokens[i - p - 1] for p in reversed(range(n - 1)))
        target_word = tokens[i]
        ngrams.append((context, target_word))
    return ngrams

In [6]:
def update(ngram_counter, context, sentence, n):
    ngrams = get_ngrams(n, tokenize(sentence))
    for ngram in ngrams:
        if ngram in ngram_counter:
            ngram_counter[ngram] += 1.0
        else:
            ngram_counter[ngram] = 1.0

        prev_words, target_word = ngram
        if prev_words in context:
            context[prev_words].append(target_word)
        else:
            context[prev_words] = [target_word]

In [7]:
def prob(ngram_counter, context, context_token, token):
    count_of_token = ngram_counter[(context_token, token)]
    count_of_context = float(len(context[context_token]))
    result = count_of_token / count_of_context
    return result

In [8]:
def random_token(context, context_token, ngram_counter):
    r = random.random()
    map_to_probs = {}
    token_of_interest = context[context_token]
    for token in token_of_interest:
        map_to_probs[token] = prob(ngram_counter, context, context_token, token)

    sum = 0
    for token in sorted(map_to_probs):
        sum += map_to_probs[token]
        if sum > r:
            return token

In [9]:
def generate_text(n, context, token_count, ngram_counter):
    context_queue = (n - 1) * ['<START>']
    result = []
    for _ in range(token_count):
        obj = random_token(context, tuple(context_queue), ngram_counter)
        result.append(obj)
        if n > 1:
            context_queue.pop(0)
            if obj == '.':
                context_queue = (n - 1) * ['<START>']
            else:
                context_queue.append(obj)
    return ' '.join(result)

In [10]:
def create_ngram_model(n, path):
    ngram_counter = {}
    context = {}
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        sentences = text.split('.')
        for sentence in sentences:
            sentence += '.'
            update(ngram_counter, context, sentence, n)
    return ngram_counter, context

In [ ]:
start = time.time()
n_gram= 3
ngram_counter, context = create_ngram_model(n_gram, 'msd.txt')
random.seed(7)
print('Generated text:')
print(generate_text(n_gram, context, 25, ngram_counter))

Generated text:
deviates scored was Indian popularity month international 09 , a one He class ODI talent tour against years - season the 2019 more for until
